# Преобразование датасета из Толоки в датасет для бенчмарка 

In [11]:
import cv2
import pandas as pd
import os


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [12]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks


In [13]:
os.listdir('/home/kirillk/datasets/OCR/ocr_test_cases_2/')

['020.jpg',
 '010.jpg',
 '003.jpg',
 '004.jpg',
 '018.jpg',
 '007.jpg',
 '011.jpg',
 '009.jpg',
 '014.jpg',
 '019.jpg',
 '008.jpg',
 '017.jpg',
 'benchmark_results.txt',
 '005.jpg',
 '015.jpg',
 '016.jpg',
 '002.jpg',
 '012.jpg',
 '021.jpg',
 '013.jpg',
 '001.jpg',
 'test-cases.csv',
 '006.jpg']

## Загрузка исходного датасета и первичные преобразования

In [14]:
df = pd.read_csv('/home/kirillk/Documents/yandex_toloka/ocr_test_2/assignments_from_pool_11-04-2022.tsv',
                sep='\t')


In [15]:
df.columns = ['filename', 'rectangles', 'orientation', 'barcode_number', 'doc_number']

In [16]:
df['filename'] = df['filename'].apply(lambda x: x.split('/')[-1])

In [17]:
df['scan_is_good'] = True

In [18]:
df.head()

,filename,rectangles,orientation,barcode_number,doc_number,scan_is_good
0,001.jpg,"{""shape"":""rectangle""\,""left"":0.068354810388805...",vertical,2810145505821,6600245672,True
1,002.jpg,"{""shape"":""rectangle""\,""left"":0.069604377927045...",vertical,2810144032564,6600245040,True
2,003.jpg,"{""shape"":""rectangle""\,""left"":0.069724263441815...",vertical,2860598110013,6600246071,True
3,004.jpg,"{""shape"":""rectangle""\,""left"":0.070041715361836...",vertical,2810145505821,6600245672,True
4,005.jpg,"{""shape"":""rectangle""\,""left"":0.065724950298564...",vertical,2810144032533,6600245039,True


In [20]:
df.shape

(16, 6)

## Парсинг столбца rectangles

In [48]:
rec = df['rectangles'][0]

In [49]:
rec = eval(rec.replace('\\', ''))

In [50]:
rec

({'shape': 'rectangle',
  'left': 0.0683548103888052,
  'top': 0.4111787479423289,
  'width': 0.17468841341350494,
  'height': 0.024840769391466178,
  'label': 'barcode'},
 {'shape': 'rectangle',
  'left': 0.2950495346342768,
  'top': 0.5170994146210197,
  'width': 0.12833416009230064,
  'height': 0.011477506392946868,
  'label': 'doc-name'},
 {'shape': 'rectangle',
  'left': 0.45141675040109425,
  'top': 0.5170264897747041,
  'width': 0.04982051170446594,
  'height': 0.007333888827847246,
  'label': 'doc-number'})

In [51]:
example = pd.DataFrame(rec).set_index('label').drop('shape', axis=1).T.unstack().to_frame().T

In [52]:
example.columns

MultiIndex([(   'barcode',   'left'),
            (   'barcode',    'top'),
            (   'barcode',  'width'),
            (   'barcode', 'height'),
            (  'doc-name',   'left'),
            (  'doc-name',    'top'),
            (  'doc-name',  'width'),
            (  'doc-name', 'height'),
            ('doc-number',   'left'),
            ('doc-number',    'top'),
            ('doc-number',  'width'),
            ('doc-number', 'height')],
           names=['label', None])

In [53]:
example.columns = ['_'.join(col) for col in example.columns]
example.columns

Index(['barcode_left', 'barcode_top', 'barcode_width', 'barcode_height',
       'doc-name_left', 'doc-name_top', 'doc-name_width', 'doc-name_height',
       'doc-number_left', 'doc-number_top', 'doc-number_width',
       'doc-number_height'],
      dtype='object')

In [54]:
new_columns_list = []

In [55]:
for col in example.columns:
    new_columns_list.append(col.split('_')[0] + '_box_' + col.split('_')[-1])
#     print(col)

In [56]:
new_columns_list

['barcode_box_left',
 'barcode_box_top',
 'barcode_box_width',
 'barcode_box_height',
 'doc-name_box_left',
 'doc-name_box_top',
 'doc-name_box_width',
 'doc-name_box_height',
 'doc-number_box_left',
 'doc-number_box_top',
 'doc-number_box_width',
 'doc-number_box_height']

In [57]:
example.columns = new_columns_list

In [58]:
example.columns = example.columns.str.replace('-', '_')

In [59]:
example.columns

Index(['barcode_box_left', 'barcode_box_top', 'barcode_box_width',
       'barcode_box_height', 'doc_name_box_left', 'doc_name_box_top',
       'doc_name_box_width', 'doc_name_box_height', 'doc_number_box_left',
       'doc_number_box_top', 'doc_number_box_width', 'doc_number_box_height'],
      dtype='object')

In [60]:
example

,barcode_box_left,barcode_box_top,barcode_box_width,barcode_box_height,doc_name_box_left,doc_name_box_top,doc_name_box_width,doc_name_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height
0,0.068355,0.411179,0.174688,0.024841,0.29505,0.517099,0.128334,0.011478,0.451417,0.517026,0.049821,0.007334


## Подготовка данных в датасете

In [61]:
rectangles_list = []

In [62]:
for rectangles in df['rectangles']:
    rectangles = eval(rectangles.replace('\\', ''))
    rectangles = pd.DataFrame(rectangles).set_index('label').drop('shape', axis=1).T.unstack().to_frame().T
    
    rectangles.columns = ['_'.join(col) for col in rectangles.columns]
    
    new_columns_list = []
    for col in rectangles.columns:
        new_columns_list.append(col.split('_')[0] + '_box_' + col.split('_')[-1])
    
    rectangles.columns = new_columns_list
    
    rectangles.columns = rectangles.columns.str.replace('-', '_')
    rectangles.columns = rectangles.columns.str.replace('name', 'type')
    rectangles_list.append(rectangles)

In [63]:
boxes_table = pd.concat(rectangles_list, ignore_index=True)

In [64]:
df = df.join(boxes_table)

### Проверка join

In [65]:
for idx, num in enumerate(df['barcode_box_left']):
    if str(num) in df['rectangles'][idx]:
        print('ok')

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


### Готовый датасет

In [66]:
df = df.drop('rectangles', axis=1)

In [67]:
df

,filename,orientation,barcode_number,doc_number,scan_is_good,barcode_box_left,barcode_box_top,barcode_box_width,barcode_box_height,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height
0,001.jpg,vertical,2810145505821,6600245672,True,0.068355,0.411179,0.174688,0.024841,0.295050,0.517099,0.128334,0.011478,0.451417,0.517026,0.049821,0.007334
1,002.jpg,vertical,2810144032564,6600245040,True,0.069604,0.410963,0.172521,0.025454,0.293708,0.516459,0.129608,0.012880,0.448891,0.516766,0.052883,0.008894
2,003.jpg,vertical,2860598110013,6600246071,True,0.069724,0.410080,0.175275,0.024133,0.292754,0.514779,0.134342,0.013366,0.454807,0.517144,0.047580,0.007628
3,004.jpg,vertical,2810145505821,6600245672,True,0.070042,0.411530,0.173070,0.023404,0.295970,0.516706,0.126862,0.012034,0.452468,0.517401,0.048909,0.006866
4,005.jpg,vertical,2810144032533,6600245039,True,0.065725,0.410182,0.174255,0.024227,0.288529,0.515065,0.133075,0.012880,0.446579,0.516449,0.050331,0.008398
5,006.jpg,vertical,2810145529968,6600245674,True,0.067505,0.410621,0.170803,0.023817,0.291121,0.516413,0.127465,0.011273,0.449622,0.517481,0.047439,0.006935
6,007.jpg,vertical,2810145529971,6600245675,True,0.062883,0.410358,0.171272,0.024956,0.287717,0.516819,0.129575,0.011894,0.445353,0.518321,0.048734,0.006992
7,008.jpg,vertical,2810144032601,6600245043,True,0.065466,0.412070,0.172929,0.022815,0.291607,0.516997,0.129393,0.011675,0.449507,0.517895,0.047482,0.006943
8,009.jpg,vertical,2810145529940,6600247340,True,0.067151,0.409830,0.177135,0.025815,0.292053,0.514969,0.133349,0.015020,0.451228,0.518357,0.049621,0.006628
9,010.jpg,vertical,2810145529996,6600245673,True,0.065800,0.410495,0.173821,0.025454,0.291205,0.515992,0.130908,0.012267,0.448120,0.517432,0.049983,0.007480


## Проверка на изображениях

In [68]:
mapping_prefix_value = {
    'barcode_box_': 'barcode_number',
    'doc_type_box_': None,
    'doc_number_box_': 'doc_number',
}

for index, row in df.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_test_cases_2/' + row['filename'])
#     image = cv2.resize(image, (0, 0), fx=0.6, fy=0.6)
    height, width, _ = image.shape

    for prefix, value in mapping_prefix_value.items():
        x1 = int(row[prefix + 'left'] * width)
        y1 = int(row[prefix + 'top'] * height)
        x2 = int((row[prefix + 'left'] + row[prefix + 'width']) * width)
        y2 = int((row[prefix + 'top'] + row[prefix + 'height']) * height)
    
        image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 4)
        image = cv2.putText(image, str(row.get(value, default='')), (x1, y2 + 50), 0, 2, (0, 0, 255), 3)
        
        cv2.imshow('Document', image)
    
        key = cv2.waitKey(2000) & 0xFF

        if key == ord('q'):
            break
    
    cv2.destroyAllWindows()

In [69]:
df['barcode_number'] = df['barcode_number'].astype('str')
df['doc_number'] = df['doc_number'].astype('str')

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   filename               16 non-null     object 
 1   orientation            16 non-null     object 
 2   barcode_number         16 non-null     object 
 3   doc_number             16 non-null     object 
 4   scan_is_good           16 non-null     bool   
 5   barcode_box_left       16 non-null     float64
 6   barcode_box_top        16 non-null     float64
 7   barcode_box_width      16 non-null     float64
 8   barcode_box_height     16 non-null     float64
 9   doc_type_box_left      16 non-null     float64
 10  doc_type_box_top       16 non-null     float64
 11  doc_type_box_width     16 non-null     float64
 12  doc_type_box_height    16 non-null     float64
 13  doc_number_box_left    16 non-null     float64
 14  doc_number_box_top     16 non-null     float64
 15  doc_numb

## Сохранение датасета

In [71]:
df.to_csv('/home/kirillk/datasets/OCR/ocr_test_cases_2/test-cases.csv')

In [72]:
test = pd.read_csv('/home/kirillk/datasets/OCR/ocr_test_cases_2/test-cases.csv')

In [73]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             16 non-null     int64  
 1   filename               16 non-null     object 
 2   orientation            16 non-null     object 
 3   barcode_number         16 non-null     int64  
 4   doc_number             16 non-null     int64  
 5   scan_is_good           16 non-null     bool   
 6   barcode_box_left       16 non-null     float64
 7   barcode_box_top        16 non-null     float64
 8   barcode_box_width      16 non-null     float64
 9   barcode_box_height     16 non-null     float64
 10  doc_type_box_left      16 non-null     float64
 11  doc_type_box_top       16 non-null     float64
 12  doc_type_box_width     16 non-null     float64
 13  doc_type_box_height    16 non-null     float64
 14  doc_number_box_left    16 non-null     float64
 15  doc_numb

In [75]:
ocr_test = pd.read_csv('/home/kirillk/PycharmProjects/ocr-benchmark/assets/dataset/test-cases.csv')

In [76]:
ocr_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             16 non-null     int64  
 1   filename               16 non-null     object 
 2   orientation            16 non-null     object 
 3   barcode_number         16 non-null     int64  
 4   doc_number             16 non-null     int64  
 5   scan_is_good           16 non-null     bool   
 6   barcode_box_left       16 non-null     float64
 7   barcode_box_top        16 non-null     float64
 8   barcode_box_width      16 non-null     float64
 9   barcode_box_height     16 non-null     float64
 10  doc_type_box_left      16 non-null     float64
 11  doc_type_box_top       16 non-null     float64
 12  doc_type_box_width     16 non-null     float64
 13  doc_type_box_height    16 non-null     float64
 14  doc_number_box_left    16 non-null     float64
 15  doc_numb